<a href="https://colab.research.google.com/github/marcelotournier/llm-t5/blob/main/llm-t5-tuning-ptbr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>